In [1]:
#google drive와 colab 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display

Coarse_label 작업

In [4]:
#movenet lighting 모델 가져오기
model_name = "movenet_lightning" #@param ["movenet_lightning", "movenet_thunder", "movenet_lightning_f16.tflite", "movenet_thunder_f16.tflite", "movenet_lightning_int8.tflite", "movenet_thunder_int8.tflite"]

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

In [5]:
#annotation할 데이터 가져오기
import os
from pathlib import Path

# 마운트시킨 데이터 셋 위치
data_path = '/content/gdrive/MyDrive/Tadasana'
classes = [path for path in Path(data_path).iterdir() if path.is_dir()]
classes

[PosixPath('/content/gdrive/MyDrive/Tadasana/Tadasana Wrong Steps'),
 PosixPath('/content/gdrive/MyDrive/Tadasana/Tadasana Right Steps')]

In [8]:
files = []
for cls in classes:
  files += [x.as_posix() for x in Path(cls).glob('**/*') if x.is_file()]
files

['/content/gdrive/MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana Wrong Step 2- Legs and Hand/20230312_105411.jpg',
 '/content/gdrive/MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana Wrong Step 2- Legs and Hand/20230312_105414(0).jpg',
 '/content/gdrive/MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana Wrong Step 2- Legs and Hand/20230312_105412(0).jpg',
 '/content/gdrive/MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana Wrong Step 2- Legs and Hand/20230312_105416(0).jpg',
 '/content/gdrive/MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana Wrong Step 2- Legs and Hand/20230312_105418(0).jpg',
 '/content/gdrive/MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana Wrong Step 2- Legs and Hand/20230312_105420.jpg',
 '/content/gdrive/MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana Wrong Step 2- Legs and Hand/20230312_105422(0).jpg',
 '/content/gdrive/MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana Wrong Step 2- Legs and Hand/20230312_105422.jpg',
 '/content/gdrive/MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana 

In [9]:
#inference하고 json으로 저장
def get_keypoints(image,
                  keypoints_with_scores,
                  output_image_height=None,
                  keypoint_threshold=0.0):
    height, width, channel = image.shape
    aspect_ratio = float(width) / height

    keypoints_all = []
    num_instances,_,_,_ = keypoints_with_scores.shape
    for id in range(num_instances):
        kpts_x = keypoints_with_scores[0,id,:,1]
        kpts_y = keypoints_with_scores[0,id,:,0]
        kpts_scores = keypoints_with_scores[0,id,:,2]
        kpts_abs_xy = np.stack(
            [width*np.array(kpts_x),height*np.array(kpts_y)],axis=-1)
        kpts_above_thrs_abs = kpts_abs_xy[kpts_scores > keypoint_threshold,: ]
        keypoints_all.append(kpts_above_thrs_abs)

    return np.concatenate(keypoints_all,axis=0)

In [10]:
#{'file path' : (17, 2) 배열} 상태로 딕셔너리에 저장
import json

keypoints = {}

for image_path in files:
    # 이미지 에러에 대한 처리
    # 이미지가 tf.io.read_file로 읽을 수 없는 타입인 경우에 대비
    try:
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image)
    except:
        print('image error : ', image_path)
        continue

    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    # 모델 인퍼런스 에러에 대한 처리
    try:
        keypoints_with_scores = movenet(input_image)
    except:
        print('model error : ', image_path)
        continue

    display_image = tf.expand_dims(image, axis=0)
    display_image = tf.cast(tf.image.resize_with_pad(
        display_image, 224, 224), dtype=tf.int32)
    output_overlay = get_keypoints(np.squeeze(display_image.numpy(), axis=0),
                                  keypoints_with_scores)

    keypoints.setdefault('/'.join(image_path.split('/')[3:]), output_overlay.tolist())

keypoints

{'MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana Wrong Step 2- Legs and Hand/20230312_105411.jpg': [[50.809146881103516,
   114.05294799804688],
  [49.25425720214844, 112.5057373046875],
  [49.51844787597656, 114.85060119628906],
  [52.24940490722656, 109.24932098388672],
  [52.886077880859375, 116.86019134521484],
  [65.67445373535156, 95.67823028564453],
  [65.04244232177734, 124.0748291015625],
  [83.36161041259766, 87.1178207397461],
  [85.93414306640625, 132.54421997070312],
  [69.74246978759766, 97.42985534667969],
  [58.73054885864258, 125.39224243164062],
  [111.95480346679688, 100.4484634399414],
  [113.16513061523438, 119.22254180908203],
  [143.78857421875, 102.64033508300781],
  [140.0562286376953, 129.58428955078125],
  [147.06253051757812, 121.13838195800781],
  [151.8115234375, 134.86334228515625]],
 'MyDrive/Tadasana/Tadasana Wrong Steps/Tadasana Wrong Step 2- Legs and Hand/20230312_105414(0).jpg': [[49.914093017578125,
   114.73656463623047],
  [49.042564392089844, 114

In [11]:
#명령어 실행 시 json 파일 생성된 것 확인 가능
with open("./keypoints.json", "w") as json_file:
    json.dump(keypoints, json_file)

In [12]:
#명령어 실행 시 로컬 PC에 파일 저장 가능
from google.colab import files
files.download("./keypoints.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
#이미지 데이터와 keypoints 데이터를 로드하는 데이터로더 만들기
import os
import pathlib

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import json

with open("./keypoints.json", "r") as json_file:
    keypoint_dict = json.load(json_file)

new_keypoint_dict = {}
for key in keypoint_dict.keys():
    new_key = '/'.join(key.split('/')[-2:])
    new_val = keypoint_dict[key]
    new_keypoint_dict[new_key] = new_val

del keypoint_dict
keypoint_dict = new_keypoint_dict
del new_keypoint_dict

def process_keypoint(file_path):
    file_path = '/'.join(file_path.numpy().decode('utf-8').split('/')[-2:])
    try:
        keypoint = tf.convert_to_tensor(keypoint_dict[file_path], dtype=tf.float32)
    except KeyError:
        print(f"KeyError: The key '{file_path}' not found in keypoint_dict.")
        # 기본 키포인트 값 설정 (예: 모든 값이 0인 배열)
        keypoint = tf.zeros([17, 2], dtype=tf.float32)  # 17x2는 예시입니다. 실제 키포인트 차원에 맞게 조정하세요.
    return keypoint


def process_path(file_path, class_names, img_shape=(224, 224)):
    label = tf.strings.split(file_path, os.path.sep)
    label = label[-2] == class_names
    label = tf.cast(label, tf.float32)

    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, img_shape)
    # img = img / 255.0

    [keypoint,] = tf.py_function(process_keypoint, [file_path], [tf.float32])

    return {"input_1": img, "input_2": keypoint}, label


def load_label(label_path):
    class_names = []
    with open(label_path) as f:
        for line in f:
            line = line.strip()
            class_names.append(line)

    return np.array(class_names)


def get_spilt_data(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):
    assert (train_split + val_split) == 1

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)

    return train_ds, val_ds


def augment(inputs, label):
    image, keypoint = inputs['input_1'], inputs['input_2']
    image = tf.image.random_crop(image, size=[224, 224, 3])
    image = tf.image.adjust_brightness(image, 0.4)
    image = tf.image.random_brightness(image, max_delta=0.4)
    return {'input_1' : image, 'input_2' : keypoint}, label


def prepare_for_training(ds, batch_size=32, cache=True, training=True):
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.repeat()
    if training:
        ds = ds.map(lambda x, y: augment(x, y))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

#load_data의 역할 : 라벨 로드, 이미지 로드, 데이터 배치 만들기 등의 역할을 하는 함수를 호출하여 최종으로 받을 데이터를 만드는 역할
def load_data(data_path, img_shape, batch_size=32, is_train=True):
    class_names = [cls for cls in os.listdir(data_path) if cls != '.DS_Store']
    data_dir = pathlib.Path(data_path)
    list_ds = tf.data.Dataset.list_files(str(data_dir / '*/*'))

    labeled_ds = list_ds.map(lambda x: process_path(x, class_names, img_shape))
    labeled_ds = prepare_for_training(labeled_ds, batch_size=batch_size, training=is_train)

    DATASET_SIZE = tf.data.experimental.cardinality(list_ds).numpy()

    return labeled_ds, DATASET_SIZE

In [17]:
# 데이터를 잘 가져오는지 확인
# 내 데이터 위치
train_data_path = '/content/gdrive/MyDrive'

train_ds, train_size = load_data(data_path=train_data_path, img_shape=(224, 224), batch_size=1)

for inputs, label in train_ds.take(1):
    print(inputs)
    print(label)

KeyError: The key '백업/2020-01-03-01-13-12-093 (1).jpg' not found in keypoint_dict.
{'input_1': <tf.Tensor: shape=(1, 224, 224, 3), dtype=float32, numpy=
array([[[[1.4272543, 1.4429406, 1.4547052],
         [1.4507136, 1.4624784, 1.4820862],
         [1.4908397, 1.5032448, 1.5300655],
         ...,
         [1.711428 , 1.711428 , 1.711428 ],
         [1.711428 , 1.711428 , 1.711428 ],
         [1.711428 , 1.711428 , 1.711428 ]],

        [[1.4518642, 1.4654996, 1.4833868],
         [1.426244 , 1.4576166, 1.4693812],
         [1.4152796, 1.4544952, 1.4586669],
         ...,
         [1.711428 , 1.711428 , 1.711428 ],
         [1.711428 , 1.711428 , 1.711428 ],
         [1.711428 , 1.711428 , 1.711428 ]],

        [[1.4364581, 1.4560659, 1.4698714],
         [1.4275744, 1.458947 , 1.4707117],
         [1.4256036, 1.4648194, 1.4726624],
         ...,
         [1.711428 , 1.711428 , 1.711428 ],
         [1.711428 , 1.711428 , 1.711428 ],
         [1.711428 , 1.711428 , 1.711428 ]],

   

In [18]:
#Multi-input 모델 만들기
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import VGG16

class YogaPose(tf.keras.Model):
    def __init__(self, num_classes=30, freeze=False):
        super(YogaPose, self).__init__()
        self.base_model = EfficientNetB0(include_top=False, weights='imagenet')
        self.keypoint = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(17, 2)),
                                              tf.keras.layers.Dense(34),])

        if freeze:
            self.base_model.trainable = False

        self.top = tf.keras.Sequential([tf.keras.layers.GlobalAveragePooling2D(name="avg_pool"),
                                        tf.keras.layers.BatchNormalization(),
                                        tf.keras.layers.Dropout(0.6, name="top_dropout")])

        self.concat = tf.keras.layers.Concatenate(axis=-1)
        self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")

    def call(self, inputs, training=True):
        image, keypoint = inputs['input_1'], inputs['input_2']
        x1 = self.base_model(image)
        x1 = self.top(x1)
        x2 = self.keypoint(keypoint)
        x = self.concat([x1, x2])
        x = self.classifier(x)
        return x

In [19]:
# 가짜 input
inputs = {'input_1':tf.ones([1, 224, 224, 3]), 'input_2':tf.ones([1, 17, 2])}
model = YogaPose(num_classes=2, freeze=True)
model(inputs)

16705208/16705208 [==============================] - 0s 0us/step


<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.39915833, 0.6008417 ]], dtype=float32)>

모델 학습

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate

# 데이터 로더 함수 (이전에 작성한 함수를 사용)
# load_data, process_path 등의 함수를 여기에 포함

# 모델 정의
def create_model(input_shape=(224, 224, 3), num_classes=10):
    # 이미지 입력
    image_input = Input(shape=input_shape, name='image_input')

    # CNN 레이어
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)

    # Keypoints 입력
    keypoints_input = Input(shape=(34,), name='keypoints_input')  # 34는 예시입니다. 실제 키포인트 차원에 맞게 조정하세요.

    # Concatenate 이미지 특징과 keypoints
    concatenated = Concatenate()([x, keypoints_input])

    # Dense 레이어
    x = Dense(128, activation='relu')(concatenated)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # 모델 생성
    model = Model(inputs=[image_input, keypoints_input], outputs=outputs)

    return model

# 모델 인스턴스 생성
model = create_model(input_shape=(224, 224, 3), num_classes=10)  # num_classes는 분류할 클래스 수에 맞게 조정

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 로드
train_data_path = '/path/to/your/dataset'  # 데이터셋 경로
train_ds, train_size = load_data(data_path=train_data_path, img_shape=(224, 224), batch_size=32)

# 모델 학습
model.fit(train_ds, epochs=10, steps_per_epoch=train_size//32)


NameError: ignored